In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt
from feature_engine import imputation
from feature_engine import encoding

# pipeline utility
from sklearn.pipeline import Pipeline

# preprocessing
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PolynomialFeatures

# hyper-parameter tuning
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# metrics to evaluate models
from sklearn.metrics import r2_score
from sklearn.metrics import max_error
from sklearn.metrics import mean_squared_error

# feature selection
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_regression

# linear models
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LogisticRegression

# support vector machines
from sklearn.svm import SVR
from sklearn.svm import LinearSVR

# neighbors models
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import RadiusNeighborsRegressor

# tree models
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import ExtraTreeRegressor

# ensemble models
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

In [3]:
house_data = pd.read_csv('../data/train.csv', index_col='Id')
X = house_data.drop(labels='SalePrice', axis=1)
Y = house_data['SalePrice']
X.shape, Y.shape

((1460, 79), (1460,))

In [4]:
clean_pipeline = Pipeline(steps=[('most_miss_cat', imputation.CategoricalImputer(variables=['Alley',
                                                                                            'FireplaceQu',
                                                                                            'PoolQC',
                                                                                            'Fence',
                                                                                            'MiscFeature'])),
                                 ('least_miss_cat', imputation.RandomSampleImputer(random_state='LotArea',
                                                                                   seed='observation',
                                                                                   variables=['GarageCond',
                                                                                              'GarageType',
                                                                                              'GarageFinish',
                                                                                              'GarageQual',
                                                                                              'BsmtExposure',
                                                                                              'BsmtFinType2',
                                                                                              'BsmtCond',
                                                                                              'BsmtQual',
                                                                                              'BsmtFinType1',
                                                                                              'MasVnrType',
                                                                                              'Electrical'])),
                                 ('least_miss_num', imputation.RandomSampleImputer(random_state='LotArea',
                                                                                   seed='observation',
                                                                                   variables=['LotFrontage',
                                                                                              'GarageYrBlt',
                                                                                              'MasVnrArea'])),
                                 ('encode', encoding.OrdinalEncoder(
                                     variables=X.select_dtypes(include=object).columns.to_list())),
                                 ('scaling', RobustScaler()),
                                 ('transform', QuantileTransformer(output_distribution='normal')),
                                 ('scale_to_range', MinMaxScaler())],
                          )

clean_pipeline.get_params()

{'memory': None,
 'steps': [('most_miss_cat',
   CategoricalImputer(variables=['Alley', 'FireplaceQu', 'PoolQC', 'Fence',
                                 'MiscFeature'])),
  ('least_miss_cat',
   RandomSampleImputer(random_state='LotArea', seed='observation',
                       variables=['GarageCond', 'GarageType', 'GarageFinish',
                                  'GarageQual', 'BsmtExposure', 'BsmtFinType2',
                                  'BsmtCond', 'BsmtQual', 'BsmtFinType1',
                                  'MasVnrType', 'Electrical'])),
  ('least_miss_num',
   RandomSampleImputer(random_state='LotArea', seed='observation',
                       variables=['LotFrontage', 'GarageYrBlt', 'MasVnrArea'])),
  ('encode',
   OrdinalEncoder(variables=['MSZoning', 'Street', 'Alley', 'LotShape',
                             'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
                             'Neighborhood', 'Condition1', 'Condition2',
                             'Bl

In [5]:
X_ = clean_pipeline.fit_transform(X, Y)
X_

array([[0.53092803, 0.51489897, 0.48546724, ..., 0.51075999, 0.49396369,
        0.50096519],
       [0.        , 0.51489897, 0.55885325, ..., 0.4571177 , 0.49396369,
        0.50096519],
       [0.53092803, 0.51489897, 0.49746612, ..., 0.51075999, 0.49396369,
        0.50096519],
       ...,
       [0.56463603, 0.51489897, 0.49154383, ..., 1.        , 0.49396369,
        0.50096519],
       [0.        , 0.51489897, 0.49746612, ..., 1.        , 0.49396369,
        0.50096519],
       [0.        , 0.51489897, 0.53143667, ..., 0.51075999, 0.49396369,
        0.50096519]])

In [6]:
X_.shape

(1460, 79)

In [7]:
train_x, test_x, train_y, test_y = train_test_split(X_, Y, test_size=0.2, random_state=48)
train_x.shape, test_x.shape, train_y.shape, test_y.shape

((1168, 79), (292, 79), (1168,), (292,))